In [1]:
#must use python 3.10 for this projcet. will also need to delete old conda envs in the morning
#This cell is a hugging face sanity check
#idea use clip to fectch t20 photos matching a user description use LLava to generate captions for 20 photos vector embeded them and compare to original prompt for similarity. 
#https://grok.com/share/c2hhcmQtMg%3D%3D_36b5ade9-4791-401a-86bf-3cf9eb29e0fa

# import torch
# import accelerate
# from transformers import AutoProcessor, LlavaForConditionalGeneration
# ### This cell is bassically a sanity check to ensure conda works. 

# # Load the model in half-precision
# model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf", torch_dtype=torch.float16, device_map="auto")
# processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf", use_fast=True)

# conversation = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "image", "url": "https://www.ilankelman.org/stopsigns/australia.jpg"},
#             {"type": "text", "text": "What is shown in this image?"},
#         ],
#     },
# ]

# inputs = processor.apply_chat_template(
#     conversation,
#     add_generation_prompt=True,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt"
# ).to(model.device, torch.float16)
# # Generate
# generate_ids = model.generate(**inputs, max_new_tokens=30)
# processor.batch_decode(generate_ids, skip_special_tokens=True)

# CLIP


In [10]:
from MemoryRetrieval import *
# getting image paths
import os
## Load LLava model
LLava_image_dir = "T20photos"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
##Model setup
llava_model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf", torch_dtype=torch.float16).to(device)
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf", use_fast=True)
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# Define the base directory where your images are stored
base_dir = './smallDataset/'
image_dir = "testing_photos/"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
## Pre compute clip embeddings based on photo library so tht dir should be all the photos
from PIL import Image 

def encode_images(image_dir):
    image_embeddings = []
    image_paths = []
    
    for img_path in os.listdir(image_dir):
        img = Image.open(os.path.join(image_dir, img_path))
        img = clip_preprocess(img).unsqueeze(0).to(device)
        with torch.no_grad():
            emb = clip_model.encode_image(img).cpu().numpy()
        image_embeddings.append(emb)
        image_paths.append(img_path)
    
    # Build Faiss index
    image_embeddings = np.vstack(image_embeddings)
    index = faiss.IndexFlatL2(image_embeddings.shape[1])
    index.add(image_embeddings)
    
    return index, image_embeddings, image_paths

# clip encodes images
clip_index, clip_embeddings, image_paths = encode_images(image_dir)


# Facial Recognition



In [13]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

def user_in_image(img_path):
    None
    

# LLava Step

In [ ]:
##test captioning and similarity
image = Image.open("Sample_image.jpg").convert("RGB")
print(caption_image(image, llava_model, processor))
print(image_similarity(caption_image(image, llava_model, processor)))

In [ ]:
## a function that utilizes caption facial recognition and clip to score and output highest scoring image
def rank_photos(prompt, n=10, prio_user=True, user_weight):
    #retrieve initial batch of photos with CLIP
    prompt_emb = clip_model.encode_text(clip.tokenize([prompt]).to(device)).cpu().numpy()
    distances, indices = clip_index.search(prompt_emb, n*2)
    clip_scores = 1 - distances[0] / np.max(distances[0]) #normalize to be between 0-1
    # LLava Captions photos and compuites a similarity score.
    llava_scores = []
    captions = []
    for idx in indicies[0]:
        img_path = os.path.join(image_dir, image_paths[idx])
        caption = get_llava_caption(img_path)
        score = caption_similarity(prompt, caption)
        llava_scores.append(score)
        captions.append(caption)
    llava_scores = np.array(llava_scores)
    llava_scores = llava_scores / np.max(llava_scores)  # Normalize
    
    # use facial recognition to computre probability of a user being in the photo
    user_present = [] #score of if user is present in photo
    for idx in indices[0]:
        img_path = os.path.join(image_dir, image_paths[idx])
        score = user_in_image(img_path) if prio_user else 0.0 #change this to whatever your function to calculate user dection is
        user_present.append(score)
    user_present = np.array(user_present)
    #score each image and output n images in descending order.
    image_scores = 0.4 * clip_scores + 0.2 * llava_scores + user_weight * me_scores
    top_n_images = np.argsort(final_scores)[::-1][:n]
    
    results = []
    for i in top_n_images:
        idx = indices[0][i]
        results.append({
            "path": image_paths[idx],
            "caption": captions[i],
            "clip_score": clip_scores[i],
            "llava_score": llava_scores[i],
            "me_score": me_scores[i],
            "final_score": final_scores[i]
        })
    return results